# <center> FullORM</center>

In [1]:
from sqlalchemy import create_engine 
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship 

import pickle

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = 'users'
    
    id     = Column(Integer, primary_key=True)
    name   = Column(String(100))  # Límite de 100 caracteres
    email  = Column(String(120), unique=True)  # Límite de 120 caracteres y es unica
    orders = relationship("Order", back_populates="user")
    # back_populates="user" significa que la clase Order 
    # también tiene una relación de vuelta hacia User.
    def __str__(self):
        return self.name + "con mail :" + self.email

In [4]:
class Product(Base):
    __tablename__ = 'products'
    
    id          = Column(Integer, primary_key=True)
    name        = Column(String(200))  # Limit to 200 characters
    price       = Column(Integer)
    #relacion 
    order_items = relationship("OrderItem", back_populates="product")
    # significa que la clase OrderItem tiene un campo que apunta de vuelta al producto.

In [5]:
class Order(Base):
    __tablename__ = 'orders'
    
    id          = Column(Integer, primary_key=True)
                # user_id Es una clave foránea (ForeignKey) que apunta al campo id de la tabla users.
    user_id     = Column(Integer, ForeignKey('users.id'))
    # Relaciones
    # Establece la relación Python entre Order y User.
    # Permite acceder al usuario que hizo el pedido: pedido.user.
    user        = relationship("User", back_populates="orders")
    order_items = relationship("OrderItem", back_populates="order") 
    #indica que en la clase OrderItem también existe un campo order que enlaza de vuelta.

In [6]:
class OrderItem(Base):
    __tablename__ = 'order_items'
    
    id         = Column(Integer, primary_key=True)
    order_id   = Column(Integer, ForeignKey('orders.id')) 
    product_id = Column(Integer, ForeignKey('products.id'))
    quantity   = Column(Integer)
    # relacones
    order   = relationship("Order", back_populates="order_items")
    product = relationship("Product", back_populates="order_items")

In [7]:
engine = create_engine('sqlite:///comercio2.db', echo=True)
Base.metadata.create_all(engine)

2025-05-26 20:48:09,533 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 20:48:09,548 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-05-26 20:48:09,562 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 20:48:09,573 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-05-26 20:48:09,577 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 20:48:09,584 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-05-26 20:48:09,589 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 20:48:09,593 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order_items")
2025-05-26 20:48:09,596 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-26 20:48:09,609 INFO sqlalchemy.engine.Engine COMMIT


Prepara una fábrica de sesiones para conectarse a la base.

session = Session() crea una sesión activa que usarás para agregar, consultar o modificar datos.

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

## Crud para la tabla User

In [9]:
# Create
def add_user(name, email):
    try:
        new_user = User(name=name, email=email)
        session.add(new_user)
        session.commit()
    except Exception as e:
        print(e)
        session.rollback()

# Read
def get_user(user_id=None):
    if user_id is not None:
        return session.query(User).filter(User.id == user_id).first()
    else:
         return session.query(User).all()
# Update
def update_user(user_id, name=None, email=None):
    user = session.query(User).filter(User.id == user_id).first()
    if user:
        if name:
            user.name = name
        if email:
            user.email = email
        session.commit()

# Delete
def delete_user(user_id):
    user = session.query(User).filter(User.id == user_id).first()
    if user:
        session.delete(user)
        session.commit()

In [24]:
add_user("emiliano"*100, "emi@ht.com")

2025-05-26 20:49:46,855 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-05-26 20:49:46,858 INFO sqlalchemy.engine.Engine [cached since 82.89s ago] ('emilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemili ... (502 characters truncated) ... lianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemiliano', 'emi@ht.com')
2025-05-26 20:49:46,865 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
usuarios = get_user()

In [12]:
for usuario in usuarios:
    print(usuario)

emilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianoemilianocon mail :emi@ht.com


In [10]:
## PRODUCT TABLE CRUD
# Create
def add_product(name, price):
    new_product = Product(name=name, price=price)
    session.add(new_product)
    session.commit()

# Read
def get_product(product_id):
    return session.query(Product).filter(Product.id == product_id).first()

# Update
def update_product(product_id, name=None, price=None):
    product = session.query(Product).filter(Product.id == product_id).first()
    if product:
        if name:
            product.name = name
        if price:
            product.price = price
        session.commit()

# Delete
def delete_product(product_id):
    product = session.query(Product).filter(Product.id == product_id).first()
    if product:
        session.delete(product)
        session.commit()

In [11]:
## ORDERS TABLE CRUD
# Create
def add_order(user_id):
    new_order = Order(user_id=user_id)
    session.add(new_order)
    session.commit()

# Read
def get_order(order_id):
    return session.query(Order).filter(Order.id == order_id).first()

# Update
def update_order(order_id, user_id=None):
    order = session.query(Order).filter(Order.id == order_id).first()
    if order:
        if user_id:
            order.user_id = user_id
        session.commit()

# Delete
def delete_order(order_id):
    order = session.query(Order).filter(Order.id == order_id).first()
    if order:
        session.delete(order)
        session.commit()

In [12]:
## ORDER ITEMS CRUD
# Create
def add_order_item(order_id, product_id, quantity):
    new_order_item = OrderItem(order_id=order_id, product_id=product_id, quantity=quantity)
    session.add(new_order_item)
    session.commit()

# Read
def get_order_item(order_item_id):
    return session.query(OrderItem).filter(OrderItem.id == order_item_id).first()

# Update
def update_order_item(order_item_id, order_id=None, product_id=None, quantity=None):
    order_item = session.query(OrderItem).filter(OrderItem.id == order_item_id).first()
    if order_item:
        if order_id:
            order_item.order_id = order_id
        if product_id:
            order_item.product_id = product_id
        if quantity:
            order_item.quantity = quantity
        session.commit()

# Delete
def delete_order_item(order_item_id):
    order_item = session.query(OrderItem).filter(OrderItem.id == order_item_id).first()
    if order_item:
        session.delete(order_item)
        session.commit()

In [13]:
def add_sample_data():
    # Adding users
    user_data = [
        ("Alice", "alice@example.com"),
        ("Bob", "bob@example.com"),
        ("Charlie", "charlie@example.com"),
        ("David", "david@example.com"),
        ("Eve", "eve@example.com"),
        ("Frank", "frank@example.com"),
        ("Grace", "grace@example.com"),
        ("Heidi", "heidi@example.com"),
        ("Ivan", "ivan@example.com"),
        ("Judy", "judy@example.com")
    ]
    for name, email in user_data:
        add_user(name, email)

    # Adding products
    product_data = [
        ("Laptop", 1000),
        ("Smartphone", 500),
        ("Tablet", 300),
        ("Monitor", 150),
        ("Keyboard", 50),
        ("Mouse", 25),
        ("Printer", 200),
        ("Router", 75),
        ("Webcam", 60),
        ("Headphones", 80)
    ]
    for name, price in product_data:
        add_product(name, price)

    # Adding orders
    for i in range(1, 11):
        add_order(i)

    # Adding order_items
    order_item_data = [
        (1, 1, 2),
        (1, 2, 1),
        (2, 3, 1),
        (2, 4, 2),
        (3, 5, 3),
        (3, 6, 1),
        (4, 7, 2),
        (4, 8, 1),
        (5, 9, 1),
        (5, 10, 2),
        (6, 1, 1),
        (6, 2, 2),
        (7, 3, 1),
        (7, 4, 1),
        (8, 5, 2),
        (8, 6, 1),
        (9, 7, 3),
        (9, 8, 1),
        (10, 9, 1),
        (10, 10, 1)
    ]
    for order_id, product_id, quantity in order_item_data:
        add_order_item(order_id, product_id, quantity)

    #Pickle this data
    # Combine data into a single dictionary
    combined_data = {
        'user_data': user_data,
        'product_data': product_data,
        'order_item_data': order_item_data
    }

    # Pickling the combined data to a single file
    with open('sample_data.pkl', 'wb') as f:
        pickle.dump(combined_data, f)

# Call the function to add sample data
add_sample_data()

2025-05-26 20:48:23,962 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 20:48:23,969 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-05-26 20:48:23,975 INFO sqlalchemy.engine.Engine [generated in 0.00627s] ('Alice', 'alice@example.com')
2025-05-26 20:48:23,979 INFO sqlalchemy.engine.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: users.email
[SQL: INSERT INTO users (name, email) VALUES (?, ?)]
[parameters: ('Alice', 'alice@example.com')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
2025-05-26 20:48:23,996 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 20:48:23,999 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-05-26 20:48:24,002 INFO sqlalchemy.engine.Engine [cached since 0.03258s ago] ('Bob', 'bob@example.com')
2025-05-26 20:48:24,005 INFO sqlalchemy.engine.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: users.email
[SQL: INSERT INTO users (name

In [14]:
def print_all_users():
    users = session.query(User).all()
    for user in users:
        print(f"User ID: {user.id}, Name: {user.name}, Email: {user.email}")

# Call the function to print all users
print_all_users()

2025-05-26 20:48:26,081 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 20:48:26,094 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users
2025-05-26 20:48:26,099 INFO sqlalchemy.engine.Engine [generated in 0.00526s] ()
User ID: 1, Name: Alice, Email: alice@example.com
User ID: 2, Name: Bob, Email: bob@example.com
User ID: 3, Name: Charlie, Email: charlie@example.com
User ID: 4, Name: David, Email: david@example.com
User ID: 5, Name: Eve, Email: eve@example.com
User ID: 6, Name: Frank, Email: frank@example.com
User ID: 7, Name: Grace, Email: grace@example.com
User ID: 8, Name: Heidi, Email: heidi@example.com
User ID: 9, Name: Ivan, Email: ivan@example.com
User ID: 10, Name: Judy, Email: judy@example.com


In [15]:
def print_all_orders():
    orders = session.query(Order).all()
    for order in orders:
        print(f"Order ID: {order.id}")
        print(f"User: {order.user.name} (ID: {order.user_id})")
        print("Order Items:")
        for item in order.order_items:
            product = item.product
            print(f"  - {product.name}: {item.quantity} x ${product.price} = ${item.quantity * product.price}")
        total = sum(item.quantity * item.product.price for item in order.order_items)
        print(f"Total: ${total}")
        print("--------------------")

# Call the function to print all orders
print_all_orders()

2025-05-26 20:48:26,164 INFO sqlalchemy.engine.Engine SELECT orders.id AS orders_id, orders.user_id AS orders_user_id 
FROM orders
2025-05-26 20:48:26,173 INFO sqlalchemy.engine.Engine [generated in 0.00933s] ()
Order ID: 1
2025-05-26 20:48:26,183 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.id = ?
2025-05-26 20:48:26,185 INFO sqlalchemy.engine.Engine [generated in 0.00244s] (1,)
User: Alice (ID: 1)
Order Items:
2025-05-26 20:48:26,197 INFO sqlalchemy.engine.Engine SELECT order_items.id AS order_items_id, order_items.order_id AS order_items_order_id, order_items.product_id AS order_items_product_id, order_items.quantity AS order_items_quantity 
FROM order_items 
WHERE ? = order_items.order_id
2025-05-26 20:48:26,204 INFO sqlalchemy.engine.Engine [generated in 0.00701s] (1,)
2025-05-26 20:48:26,213 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.name AS products_name, pr

In [16]:
def print_all_products():
    products = session.query(Product).all()
    print("All Products:")
    print("-------------")
    for product in products:
        print(f"ID: {product.id}")
        print(f"Name: {product.name}")
        print(f"Price: ${product.price}")
        print("-------------")

# Call the function to print all products
print_all_products()

2025-05-26 20:48:26,694 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.name AS products_name, products.price AS products_price 
FROM products
2025-05-26 20:48:26,699 INFO sqlalchemy.engine.Engine [generated in 0.00467s] ()
All Products:
-------------
ID: 1
Name: Laptop
Price: $1000
-------------
ID: 2
Name: Smartphone
Price: $500
-------------
ID: 3
Name: Tablet
Price: $300
-------------
ID: 4
Name: Monitor
Price: $150
-------------
ID: 5
Name: Keyboard
Price: $50
-------------
ID: 6
Name: Mouse
Price: $25
-------------
ID: 7
Name: Printer
Price: $200
-------------
ID: 8
Name: Router
Price: $75
-------------
ID: 9
Name: Webcam
Price: $60
-------------
ID: 10
Name: Headphones
Price: $80
-------------
ID: 11
Name: Laptop
Price: $1000
-------------
ID: 12
Name: Smartphone
Price: $500
-------------
ID: 13
Name: Tablet
Price: $300
-------------
ID: 14
Name: Monitor
Price: $150
-------------
ID: 15
Name: Keyboard
Price: $50
-------------
ID: 16
Name: Mouse
Price: $

## Consultas complejas

In [17]:
from sqlalchemy.orm import joinedload

# Función para recuperar todos los pedidos de un usuario en particular junto con los artículos en cada pedido
def get_orders_by_user(user_id):
    """
    Recupera todos los pedidos realizados por un usuario específico,
    incluyendo los ítems de cada pedido y los productos asociados a esos ítems.

    Parámetros:
    -----------
    user_id : int
        ID del usuario cuyos pedidos se desean consultar.

    Retorna:
    --------
    list[Order] o None
        Lista de objetos Order si el usuario existe, o None si no se encontró.
        Cada pedido incluye sus order_items y cada ítem su product correspondiente.
    """
    user = (session.query(User)
            # Carga anticipada de la relación User.orders
            .options(joinedload(User.orders)
                      # Carga anticipada de Order.order_items
                     .joinedload(Order.order_items)
                     # Carga anticipada de OrderItem.product
                     .joinedload(OrderItem.product))
             # Filtro por ID de usuario
            .filter(User.id == user_id).first())
    if user:
        # Devuelve los pedidos si se encontró el usuario
        return user.orders
    else:
        return None

# Example usage
user_id = 5  # eemplace con el ID del usuario para el cual desea recuperar pedidos
orders = get_orders_by_user(user_id)
if orders:
    for order in orders:
        print(f"Order ID: {order.id}")
        for item in order.order_items:
            print(f"\033[1m    Product: {item.product.name}, Quantity: {item.quantity}\033[0m")
else:
    print("No se encontraron pedidos para este usuario..")

2025-05-26 20:48:27,987 INFO sqlalchemy.engine.Engine SELECT anon_1.users_id AS anon_1_users_id, anon_1.users_name AS anon_1_users_name, anon_1.users_email AS anon_1_users_email, products_1.id AS products_1_id, products_1.name AS products_1_name, products_1.price AS products_1_price, order_items_1.id AS order_items_1_id, order_items_1.order_id AS order_items_1_order_id, order_items_1.product_id AS order_items_1_product_id, order_items_1.quantity AS order_items_1_quantity, orders_1.id AS orders_1_id, orders_1.user_id AS orders_1_user_id 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.id = ?
 LIMIT ? OFFSET ?) AS anon_1 LEFT OUTER JOIN orders AS orders_1 ON anon_1.users_id = orders_1.user_id LEFT OUTER JOIN order_items AS order_items_1 ON orders_1.id = order_items_1.order_id LEFT OUTER JOIN products AS products_1 ON products_1.id = order_items_1.product_id
2025-05-26 20:48:28,005 INFO sqlalchemy.engine.Engine [generated in

In [18]:
from sqlalchemy import func

def get_total_quantity_sold():
    """
    Consulta los productos más vendidos y la cantidad total vendida de cada uno.

    Retorna:
    --------
    list[tuple[int, str, int]]
        Lista de tuplas con: (id del producto, nombre del producto, cantidad total vendida),
        ordenados de mayor a menor.
    """
    query = (session.query(Product.id, 
                           Product.name, 
                           # Suma total de unidades vendidas por producto
                           func.sum(OrderItem.quantity).label('total_vendido')) # Suma total de unidades vendidas por producto
             # Une con los ítems de pedido por ID de producto
             .join(OrderItem, Product.id == OrderItem.product_id)
             # Agrupa por producto
             .group_by(Product.id, Product.name)
             # Ordena de mayor a menor
             .order_by(func.sum(OrderItem.quantity).desc()))
    
    results = query.all() # Ejecuta la consulta y devuelve todos los resultados
    
    return [
        {
            'product_id': result.id,
            'product_nombre': result.name,
            'total_vendido': result.total_vendido
        } for result in results
    ]

# Function to print the results
def print_total_quantity_sold():
    results = get_total_quantity_sold()
    print("Total Quantity Sold for Each Product:")
    print("-------------------------------------")
    for item in results:
        print(f"Product ID: {item['product_id']}")
        print(f"Nombre Producto: {item['product_nombre']}")
        print(f"Total Vendido: {item['total_vendido']}")
        print("-------------------------------------")

# Call the function to print the results
print_total_quantity_sold()

2025-05-26 20:48:28,918 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.name AS products_name, sum(order_items.quantity) AS total_vendido 
FROM products JOIN order_items ON products.id = order_items.product_id GROUP BY products.id, products.name ORDER BY sum(order_items.quantity) DESC
2025-05-26 20:48:28,930 INFO sqlalchemy.engine.Engine [generated in 0.01151s] ()
Total Quantity Sold for Each Product:
-------------------------------------
Product ID: 5
Nombre Producto: Keyboard
Total Vendido: 10
-------------------------------------
Product ID: 7
Nombre Producto: Printer
Total Vendido: 10
-------------------------------------
Product ID: 1
Nombre Producto: Laptop
Total Vendido: 6
-------------------------------------
Product ID: 2
Nombre Producto: Smartphone
Total Vendido: 6
-------------------------------------
Product ID: 4
Nombre Producto: Monitor
Total Vendido: 6
-------------------------------------
Product ID: 10
Nombre Producto: Headphones
Total Vendido

# Almacenamiento en Cache

## Opción 1: Caché en memoria (simple, por código)

```python
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from functools import lru_cache

engine = create_engine("sqlite:///mi_base.db")
Session = sessionmaker(bind=engine)
session = Session()

@lru_cache(maxsize=128)
def get_usuario_por_id(usuario_id):
    return session.query(Usuario).filter_by(id=usuario_id).first()
```

Esto mantiene una copia en memoria (por proceso) de hasta 128 resultados de la función.

## Opción 2: Usar un sistema de caché como Redis

Puedes guardar en Redis los resultados de ciertas consultas costosas:

```python
import redis
import pickle

r = redis.Redis(host='localhost', port=6379, db=0)

def get_usuario_por_id(usuario_id):
    cache_key = f"usuario:{usuario_id}"
    cached = r.get(cache_key)
    if cached:
        return pickle.loads(cached)

    usuario = session.query(Usuario).filter_by(id=usuario_id).first()
    if usuario:
        r.setex(cache_key, 300, pickle.dumps(usuario))  # 5 minutos de cache
    return usuario

```

Redis es un servicio que corre en tu máquina o en un servidor, como si fuera una base de datos en memoria.

- 🔹 En Ubuntu/Debian:
```bash
sudo apt update
sudo apt install redis-server
```

- 🔹 En macOS (con Homebrew):

```bash
brew install redis
brew services start redis
```

- 🔹 En Windows:
Redis no tiene soporte oficial, pero podés:

Usar WSL (Linux en Windows) y seguir el paso de Ubuntu.

O instalarlo con Docker:

```bash
docker run -d -p 6379:6379 redis
```

Una vez instalado, podés verificar que funciona con:

```bash
redis-cli ping
```

Debe responder: PONG

- ✅ 2. Instalar la librería cliente de Redis en Python

```bash
pip install redis
```

- 🧪 Prueba simple de conexión

```python
import redis

r = redis.Redis(host='localhost', port=6379, db=0)
r.set("clave", "valor")
print(r.get("clave"))  # b'valor'
```

## Uso de dogpile.cache

Dogpile.cache es una excelente opción para usar caché en Python de forma más estructurada y con soporte para múltiples backends (memoria, archivo, Redis, Memcached, etc.). Funciona muy bien junto a SQLAlchemy, y de hecho es la base del sistema de caché opcional de SQLAlchemy.

- ✅ ¿Qué es Dogpile.cache?
Es una biblioteca que gestiona cachés con expiración, y permite que solo una llamada recalcule un valor cuando el caché expira ("dogpile locking"). Es ideal para evitar que muchas consultas golpeen la base de datos innecesariamente.

- 🔧 Instalación

```bash
pip install dogpile.cache
```

- 🚀 Ejemplo básico con SQLAlchemy

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dogpile.cache import make_region
import json

# Configura SQLAlchemy
engine = create_engine("sqlite:///mi_base.db")
Session = sessionmaker(bind=engine)
session = Session()

# Configura el caché (en memoria)
region = make_region().configure(
    'dogpile.cache.memory',
    expiration_time=60  # 1 minuto de expiración
)

# Suponiendo que tienes una clase Usuario
def load_usuario(usuario_id):
    print("→ Consultando base de datos...")
    usuario = session.query(Usuario).filter_by(id=usuario_id).first()
    return usuario

@region.cache_on_arguments()
def get_usuario_cacheado(usuario_id):
    return load_usuario(usuario_id)
```

- 🧪 Uso

```python
# Primera vez: consulta la base
usuario = get_usuario_cacheado(1)

# Segunda vez (dentro de 60 segundos): viene del caché
usuario = get_usuario_cacheado(1)
```

Verás que solo se imprime "→ Consultando base de datos..." una vez.

- 📦 Backends disponibles

Puedes cambiar dogpile.cache.memory por otros:

- 'dogpile.cache.redis'
- 'dogpile.cache.memcached'
- 'dogpile.cache.dbm' (caché en archivo)
- 'dogpile.cache.memory_pickle'

Por ejemplo, para usar Redis:

```python
region = make_region().configure(
    'dogpile.cache.redis',
    arguments = {
        'host': 'localhost',
        'port': 6379,
        'db': 0,
        'redis_expiration_time': 60,
    }
)
```

## Practica de cache

In [19]:
import time
from dogpile.cache import make_region
from sqlalchemy.orm import Session

In [20]:
# Configure the cache region
region = make_region().configure(
    'dogpile.cache.memory',
    expiration_time=3600
)


In [21]:
@region.cache_on_arguments()
def get_all_products_cached():
    with Session(engine) as session:
        query = session.query(Product)
        return [
            {
                'id': product.id,
                'name': product.name,
                'price': product.price
            } for product in query.all()
        ]

In [22]:
def get_all_products_uncached():
    with Session(engine) as session:
        query = session.query(Product)
        return [
            {
                'id': product.id,
                'name': product.name,
                'price': product.price
            } for product in query.all()
        ]

In [23]:
def compare_execution_times(num_iterations=5):
    print(f"Comparación de los tiempos de ejecución a lo largo de {num_iterations} iteraciones:")
    
    # Warm up the cache
    get_all_products_cached()
    
    # Test cached function
    cache_start_time = time.time()
    for _ in range(num_iterations):
        get_all_products_cached()
    cache_end_time = time.time()
    cache_total_time = cache_end_time - cache_start_time
    
    # Test uncached function
    uncache_start_time = time.time()
    for _ in range(num_iterations):
        get_all_products_uncached()
    uncache_end_time = time.time()
    uncache_total_time = uncache_end_time - uncache_start_time
    
    print(f"Tiempo total de la función en caché: {cache_total_time:.6f} segundos")
    print(f"Tiempo promedio de la función en caché: {cache_total_time/num_iterations:.6f} segundos")
    print(f"Tiempo total de función no almacenada en caché: {uncache_total_time:.6f} segundos")
    print(f"Tiempo promedio de función no almacenada en caché: {uncache_total_time/num_iterations:.6f} segundos")
    print(f"Mejora de la velocidad: {uncache_total_time/cache_total_time:.2f}x más rápido con almacenamiento en caché")

# Run the comparison
compare_execution_times()

Comparación de los tiempos de ejecución a lo largo de 5 iteraciones:
2025-05-26 20:48:41,978 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 20:48:41,980 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.name AS products_name, products.price AS products_price 
FROM products
2025-05-26 20:48:41,983 INFO sqlalchemy.engine.Engine [cached since 15.29s ago] ()
2025-05-26 20:48:41,986 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-26 20:48:41,990 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 20:48:41,993 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.name AS products_name, products.price AS products_price 
FROM products
2025-05-26 20:48:41,995 INFO sqlalchemy.engine.Engine [cached since 15.3s ago] ()
2025-05-26 20:48:41,998 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-26 20:48:42,001 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-26 20:48:42,004 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, 